In [109]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
from corsikaio import CorsikaFile
import re
import os
import matplotlib.animation as animation

In [102]:
filename = 'pDAT143000'
metadata_filename = 'pDAT143000.dbase'
file_path = f'data/p30/{filename}' 
metadata_file_path = f'data/p30/{metadata_filename}' 

save_path = 'csv_otput/pDAT143000_output' 

In [104]:
os.makedirs(save_path, exist_ok=True) 

In [105]:
def parse_metadata(file_path):
    metadata = {}

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip() 

            match = re.search(r'#howmanyshowers#\s*(\d+)', line)
            if match:
                metadata['howmanyshowers'] = int(match.group(1))

            match = re.search(r'#energy_prim#\s*([\d\.eE\+\-]+)', line)
            if match:
                metadata['energy_prim'] = float(match.group(1))

            match = re.search(r'#theta_prim#\s*([\d\.eE\+\-]+)', line)
            if match:
                metadata['theta_prim'] = float(match.group(1))

            match = re.search(r'#phi_prim#\s*([\d\.eE\+\-]+)', line)
            if match:
                metadata['phi_prim'] = float(match.group(1))

            match = re.search(r'#dsn_events#\s*(\S+)', line)
            if match:
                metadata['dsn_events'] = match.group(1)

    return metadata

In [106]:
PID_MAP = {
    1: "gamma", 2: "e-", 3: "e+", 4: "ν_e", 5: "μ-", 6: "μ+",
    7: "π⁰", 8: "π+", 9: "π-", 13: "n", 14: "p", 75: "Fe",
    25: "Λ⁰", 35: "Σ⁺", 45: "Ξ⁰", 55: "Ω⁻", 65: "anti-proton",
}

def pid_name(pid_raw):
    """Преобразует 'сырой' PID в читаемое имя"""
    base_pid = pid_raw // 1000
    return PID_MAP.get(base_pid, f"unknown({base_pid})")


In [ ]:
def save_event_data(event_idx, particles, header, save_path):
    theta_prim = header.get('theta_prim', 0)
    phi_prim = header.get('phi_prim', 0)
    energy_prim = header.get('energy_prim', 0)
    
    event_data = []

    has_time = particles.shape[1] > 7 

    for p in particles:
        raw_pid = int(p[0])
        base_pid = raw_pid // 1000
        name = pid_name(raw_pid)
        
        px, py, pz = p[1], p[2], p[3]
        x, y, z = p[4] / 100, p[5] / 100, p[6] 

        energy = np.sqrt(px**2 + py**2 + pz**2)
        
        time = p[7] if has_time else None
        event_data.append({
            'event_id': event_idx,
            'particle_name': name,
            'pid': base_pid,
            'x': x,
            'y': y,
            'z': z,
            'energy': energy,
            'theta_prim': theta_prim,
            'phi_prim': phi_prim,
            'energy_prim': energy_prim,
            'time': time 
        })
    

In [114]:
with CorsikaFile(file_path) as f:
    for event_idx, event in enumerate(f):
        header = event.header
        particles = event.data
        
        metadata = parse_metadata(metadata_file_path) 
        if isinstance(header, np.void):
            header = {field: header[field] for field in header.dtype.names}
        
        header.update(metadata) 
        save_event_data(event_idx, particles, header, save_path)